## preprocessing data

In [3]:
import scipy.io
import numpy as np
import pandas as pd


import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.linear_model import LogisticRegression, LinearRegression

from matplotlib import pyplot as plt


In [4]:
def get_all_fft(filepath):
    mat = scipy.io.loadmat(filepath)
    all_fft = mat['all_fft']
    return all_fft

def get_entropy(all_fft):
    mat_dict = {'fft_band_mean':0,
        'fft_band_array':1,
        'fft_band_smooth_mean':2,
        'fft_band_smooth_array':3, 
        'chan':4,
        'file':5,
        'task':6,
        'time':7,
        'raw':8, 
        'filter_data_array':9, 
        'psd_ratio_array':10}
    
    #different parameters
    chan_list = []
    subject_list = []
    task_list = []
    trial_index_list= []
    task_type_list = []
    task_type_idx_list = []
    day_list = []
      
        
    #observed variables: fft magnitue, power ratio
    entropy_list = []     
    df = pd.DataFrame(columns=['chan','subject','task','entropy'])   
    trial_index = 0
    trial_index_start = 0
       
    for i in range(all_fft.shape[1]):
        if i == 0:
            pre_trial = all_fft[0,i][mat_dict['task']][0]
            pre_sub = all_fft[0,i][mat_dict['file']][0]

        curr_trial = all_fft[0,i][mat_dict['task']][0]
        curr_sub = all_fft[0,i][mat_dict['file']][0]

        if pre_trial == curr_trial and pre_sub==curr_sub: 
            trial_index_start = trial_index
            #same subject and same task, not change the trial_index label
            pass
        else:
            trial_index = trial_index_start

        pre_trial = curr_trial
        pre_sub = curr_sub
                
        all_ts = all_fft[0,i][mat_dict['filter_data_array']][0]
        step = 50       
        print(i)
        
        
        chan = [all_fft[0,i][mat_dict['chan']][0][0]]
        day = [all_fft[0,i][mat_dict['time']][0]]
        subject = [all_fft[0,i][mat_dict['file']][0]]
        task = [all_fft[0,i][mat_dict['task']][0]]
        task_type = [all_fft[0,i][mat_dict['task']][0][0:-1]]
        task_type_index = [all_fft[0,i][mat_dict['task']][0]]
        
        for j in range(0,len(all_ts)-250,step):
            ts = all_ts[j:j+249]
            std_ts = np.std(ts)
            sample_entropy = ent.sample_entropy(ts, 3, 0.25*std_ts)[2]
            
            entropy_list.append(sample_entropy)            
            chan_list.extend(chan) #channel
            subject_list.extend(subject)
            task_list.extend(task)                     
            trial_index_list.append(trial_index_start)
            trial_index_start = trial_index_start + 1
            
            task_type_list.extend(task_type)
            task_type_idx_list.extend(task_type_index)   
            day_list.append(day)

    df['chan'] = chan_list
    df['day']=day_list
    df['task'] = task_list
    df['subject'] = subject_list
    df['trial_index']=trial_index_list
    df['trial_type'] = task_type_list
    df['trial_type_idx'] = task_type_idx_list    
    df['entropy']= entropy_list
    
    return df
    
    
    
    
#export the pandas dataframe from the all_fft data
def all_fft_to_df(all_fft):

    #sub: chosen subject list
    #val: fft magnitude, 0 v.s. power ratio, 1, default: fft magnitude
    
    #the column of the mat file
    mat_dict = {'fft_band_mean':0,
            'fft_band_array':1,
            'fft_band_smooth_mean':2,
            'fft_band_smooth_array':3, 
            'chan':4,
            'file':5,
            'task':6,
            'time':7,
            'raw':8, 
            'filter_data_array':9, 
            'psd_ratio_array':10}

    
    df = pd.DataFrame(columns=['chan','subject','task','freq','fft_band_smooth', 'psd_ratio'])

    #different parameters
    chan_list = []
    subject_list = []
    task_list = []
    freq_list = []
    trial_index_list= []
    task_type_list = []
    task_type_idx_list = []
    day_list = []
    
    #observed variables: fft magnitue, power ratio
    fft_list = []
    raw_fft_list = []
    psd_list = []
    

    trial_index_start = 0

    for i in range(all_fft.shape[1]):
        if i == 0:
            pre_trial = all_fft[0,i][mat_dict['task']][0]
            pre_sub = all_fft[0,i][mat_dict['file']][0]


        curr_trial = all_fft[0,i][mat_dict['task']][0]
        curr_sub = all_fft[0,i][mat_dict['file']][0]


        if pre_trial == curr_trial and pre_sub==curr_sub:        
            #same subject and same task, not change the trial_index label
            pass
        else:
            trial_index_start = trial_index_start+len_fft

        pre_trial = curr_trial
        pre_sub = curr_sub
        
        raw_fft = all_fft[0,i][mat_dict['fft_band_array']][[0,1,2,4],1:] #first two row: alpha, beta1, beta2, theta
        fft = all_fft[0,i][mat_dict['fft_band_smooth_array']][[0,1,2,4],1:] #first two row: alpha, beta1, beta2, theta
        psd = all_fft[0,i][mat_dict['psd_ratio_array']][:,1:] #first two row: alpha, beta
        len_fft = fft.shape[1]

        chan = [all_fft[0,i][mat_dict['chan']][0][0]] * len_fft
        day = [all_fft[0,i][mat_dict['time']][0]] * len_fft
        subject = [all_fft[0,i][mat_dict['file']][0]] *len_fft
        task = [all_fft[0,i][mat_dict['task']][0]] *len_fft
        task_type = [all_fft[0,i][mat_dict['task']][0][0:-1]] *len_fft
        task_type_index = [all_fft[0,i][mat_dict['task']][0]] *len_fft

        trial_index = range(trial_index_start, trial_index_start+len_fft)

        for j in range(4):               
            freq = [j]*len_fft

            chan_list.extend(chan) #channel
            subject_list.extend(subject)
            task_list.extend(task)      
            freq_list.extend(freq)                  
            trial_index_list.extend(trial_index)
            task_type_list.extend(task_type)
            task_type_idx_list.extend(task_type_index)
            day_list.extend(day)
            
            fft_list.extend(fft[j,:]) 
            raw_fft_list.extend(raw_fft[j,:])
            psd_list.extend(psd[j,:])


    df['chan'] = chan_list
    df['day'] = day_list
    df['task'] = task_list
    df['subject'] = subject_list
    df['freq']=freq_list  
    df['trial_index']=trial_index_list
    df['trial_type'] = task_type_list
    df['trial_type_idx'] = task_type_idx_list
    
    df['fft_band_smooth']= fft_list 
    df['fft_band_raw']=  raw_fft_list
    df['psd_ratio'] = psd_list
    
    return df


def get_feature_from_df(df, sub, task_list,select_chan, val=0):  
    select_chan_freq = []
    for i in select_chan:
        for j in range(4):
            select_chan_freq.append(str(i)+str(j))


    if val == 0: #fft
        df_select = df[df['subject'].isin(sub)][['chan', 'freq', 'fft_band_smooth', 'trial_index',
                                                 'trial_type', 'trial_type_idx','day']]
        df_select['chanfreq'] = df_select['chan'].apply(str)+ df_select['freq'].apply(str)
        df_feature_pivot = df_select.pivot(index='trial_index', columns='chanfreq', values='fft_band_smooth')
        df_class_pivot = df_select.pivot(index='trial_index', columns='chanfreq', values='trial_type')  
        df_day_pivot = df_select.pivot(index='trial_index', columns='chanfreq', values='day')  
        
        X = df_feature_pivot[select_chan_freq]
        y, uniques = pd.factorize(df_class_pivot['10'],sort = True)
        day, uniques_day = pd.factorize(df_day_pivot['10'],sort = True)
        
    elif val == 1: #power ratio
        df_select = df[df['subject'].isin(sub)][['chan', 'freq', 'psd_ratio', 'trial_index',
                                                 'trial_type', 'trial_type_idx','day']]
        df_select['chanfreq'] = df_select['chan'].apply(str)+ df_select['freq'].apply(str)
        df_feature_pivot = df_select.pivot(index='trial_index', columns='chanfreq', values='psd_ratio')
        df_class_pivot = df_select.pivot(index='trial_index', columns='chanfreq', values='trial_type')
        df_day_pivot = df_select.pivot(index='trial_index', columns='chanfreq', values='day')
                
        
        X = df_feature_pivot[select_chan_freq]
        y, uniques = pd.factorize(df_class_pivot['10'],sort = True)
        day, uniques_day = pd.factorize(df_day_pivot['10'],sort = True)

              
    elif val == 2: #entropy
        df_select = df[df['subject'].isin(sub)][['chan', 'entropy', 'trial_index','trial_type', 'trial_type_idx']]
        df_select['chan'] = df_select['chan'].apply(str)
        df_feature_pivot = df_select.pivot(index='trial_index', columns='chan', values='entropy')
        df_class_pivot = df_select.pivot(index='trial_index', columns='chan', values='trial_type')
        
        X = df_feature_pivot[['1', '2', '3','4']]
        y, uniques = pd.factorize(df_class_pivot['1'],sort = True)
        
                
    elif val == 3: #raw fft
        df_select = df[df['subject'].isin(sub)][['chan', 'freq', 'fft_band_raw', 'trial_index','trial_type', 'trial_type_idx']]
        df_select['chanfreq'] = df_select['chan'].apply(str)+ df_select['freq'].apply(str)
        df_feature_pivot = df_select.pivot(index='trial_index', columns='chanfreq', values='fft_band_raw')
        df_class_pivot = df_select.pivot(index='trial_index', columns='chanfreq', values='trial_type')  
        
        X = df_feature_pivot[select_chan_freq]
        y, uniques = pd.factorize(df_class_pivot['10'],sort = True)
                
        
    else:
        print('please choose power ratio or fft magnitude or entropy for analysis')
        
    
    x_select = X.iloc[[i for i, v in enumerate(uniques.take(y)) if v in task_list],:]
    y_select = y[[i for i, v in enumerate(uniques.take(y)) if v in task_list]]
    day_select = day[[i for i, v in enumerate(uniques.take(y)) if v in task_list]]
        

    return x_select, y_select, uniques, day

    #X, the feature of the channels and alpha, beta amplitude
    #y, the target of the task; 'L1', 'L2', 'L3', 'L4', 'eyeclose', 'eyeopen'


In [5]:
def smooth_raw_data(data):
    #normalize data
    #electrode-wise exponential moving standardization  
    
    #input: data, with shape (1, time)
    #first 1000 sample as average
    m = np.mean(data[:,0:1000], axis = 1) #m0 is the first 1000 datapoints mean values
    v = np.var(data[:,0:1000],axis = 1) #v0 is the first 1000 datapoints variance values
    normalize_data = np.zeros((data.shape[0], data.shape[1]))

    for i in range(1000,data.shape[1]):
        m = 0.001*data[:,i] + 0.999*m
        v = 0.001*((data[:,i]-m)**2) + 0.999*v
        normalize_data[:,i] = (data[:,i]-m)/np.sqrt(v)
        
    return normalize_data

def get_raw_data(all_fft, sub, select_task_list, day):
    mat_dict = {'fft_band_mean':0,
        'fft_band_array':1,
        'fft_band_smooth_mean':2,
        'fft_band_smooth_array':3, 
        'chan':4,
        'file':5,
        'task':6,
        'time':7,
        'raw':8, 
        'filter_data_array':9, 
        'psd_ratio_array':10}

    chan_list = []
    data_list = []
    task_list = []
    task_type_list = []

    for i in range(all_fft.shape[1]):
        curr_sub = all_fft[0,i][mat_dict['file']][0]
        task_type = all_fft[0,i][mat_dict['task']][0][0:-1]
        chan = all_fft[0,i][mat_dict['chan']][0][0]
        task = all_fft[0,i][mat_dict['task']][0]

        if curr_sub in sub and task_type in select_task_list and chan in [2,4]:
            #update calculation in every 50 sample (with 250hz, that is every 1/5 second)
            all_ts = all_fft[0,i][mat_dict['filter_data_array']]
            chan_list.append(chan)
            data_list.append(all_ts)
            task_list.append(task)
            task_type_list.append(task_type)

    df = pd.DataFrame({'chan':chan_list,
                      'data': data_list,
                      'task': task_list,
                      'task_type':task_type_list})


    X = []
    y = []
    day_list = []
    step = 50

    for tsk in np.unique(task_list):
        tmp1 = df[(df['chan'] == 2) & (df['task'] == tsk)]['data'].values[0]
        tmp2 = df[(df['chan'] == 4) & (df['task'] == tsk)]['data'].values[0]
        tmp3 = np.concatenate([tmp1, tmp2], axis = 0)
        tmp4 = smooth_raw_data(tmp3) #normalized filter data
        for i in range(1000, tmp4.shape[1]-251,step):
            ts = tmp4[:, i:i+250]
            X.append(ts)
            y.append(tsk[:-1])
            day_list.append(day)


    X = np.stack([x for x in X], axis = 0) 
    
    return X,y,day_list

In [6]:
def get_feature_from_df(df, sub, task_list,select_chan, val=0):  
    select_chan_freq = []
    for i in select_chan:
        for j in range(4):
            select_chan_freq.append(str(i)+str(j))


    if val == 0: #fft
        df_select = df[df['subject'].isin(sub)][['chan', 'freq', 'fft_band_smooth', 'trial_index',
                                                 'trial_type', 'trial_type_idx','day']]
        df_select['chanfreq'] = df_select['chan'].apply(str)+ df_select['freq'].apply(str)
        df_feature_pivot = df_select.pivot(index='trial_index', columns='chanfreq', values='fft_band_smooth')
        df_class_pivot = df_select.pivot(index='trial_index', columns='chanfreq', values='trial_type')  
        df_day_pivot = df_select.pivot(index='trial_index', columns='chanfreq', values='day')  
        
        X = df_feature_pivot[select_chan_freq]
        y, uniques = pd.factorize(df_class_pivot['10'],sort = True)
        day,uniques_day = pd.factorize(df_day_pivot['10'],sort = True)
        
    elif val == 1: #power ratio
        df_select = df[df['subject'].isin(sub)][['chan', 'freq', 'psd_ratio', 'trial_index',
                                                 'trial_type', 'trial_type_idx','day']]
        df_select['chanfreq'] = df_select['chan'].apply(str)+ df_select['freq'].apply(str)
        df_feature_pivot = df_select.pivot(index='trial_index', columns='chanfreq', values='psd_ratio')
        df_class_pivot = df_select.pivot(index='trial_index', columns='chanfreq', values='trial_type')
        df_day_pivot = df_select.pivot(index='trial_index', columns='chanfreq', values='day')
                
        
        X = df_feature_pivot[select_chan_freq]
        y, uniques = pd.factorize(df_class_pivot['10'],sort = True)
        day,uniques_day = pd.factorize(df_day_pivot['10'],sort = True)

              
    elif val == 2: #entropy
        df_select = df[df['subject'].isin(sub)][['chan', 'entropy', 'trial_index','trial_type', 'trial_type_idx']]
        df_select['chan'] = df_select['chan'].apply(str)
        df_feature_pivot = df_select.pivot(index='trial_index', columns='chan', values='entropy')
        df_class_pivot = df_select.pivot(index='trial_index', columns='chan', values='trial_type')
        
        X = df_feature_pivot[['1', '2', '3','4']]
        y, uniques = pd.factorize(df_class_pivot['1'],sort = True)
        
                
    elif val == 3: #raw fft
        df_select = df[df['subject'].isin(sub)][['chan', 'freq', 'fft_band_raw', 'trial_index','trial_type', 'trial_type_idx']]
        df_select['chanfreq'] = df_select['chan'].apply(str)+ df_select['freq'].apply(str)
        df_feature_pivot = df_select.pivot(index='trial_index', columns='chanfreq', values='fft_band_raw')
        df_class_pivot = df_select.pivot(index='trial_index', columns='chanfreq', values='trial_type')  
        
        X = df_feature_pivot[select_chan_freq]
        y, uniques = pd.factorize(df_class_pivot['10'],sort = True)
                
        
    else:
        print('please choose power ratio or fft magnitude or entropy for analysis')
        
    
    x_select = X.iloc[[i for i, v in enumerate(uniques.take(y)) if v in task_list],:]
    y_select = y[[i for i, v in enumerate(uniques.take(y)) if v in task_list]]
    day_select = day[[i for i, v in enumerate(uniques.take(y)) if v in task_list]]
        

    return x_select, y_select, uniques, day

## prepare train and test data (3 level, for 8 days)

In [7]:
#all 5 levels 
all_fft = get_all_fft('C:\\Users\\OWNER\\Downloads\\BCI_MATH_ALL_FFT\\level3_5freq_step250_all_fft.mat')
df = all_fft_to_df(all_fft)

X_train, y_train, uniques, day = get_feature_from_df(df, ['fzb'],['l1','l3','l5'], [2,4],0)

X_train['y'] = y_train()
X_train['day']=day
X_train.to_csv('D:/BCI_Classification/bci_fft_5freq_data_train.csv')


In [8]:
np.unique(X_train['day'])

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int64)

In [34]:
all_fft9_1 = get_all_fft('C:\\Users\\OWNER\\Downloads\\BCI_MATH_ALL_FFT\\20200227_level3_sameday_5freq_step250_all_fft_0.mat')
df_9_1 = all_fft_to_df(all_fft9_1)

all_fft9_2 = get_all_fft('C:\\Users\\OWNER\\Downloads\\BCI_MATH_ALL_FFT\\20200227_level3_sameday_5freq_step250_all_fft_2.mat')
df_9_2 = all_fft_to_df(all_fft9_2)

X_train, y_train, uniques_9, day = get_feature_from_df(df_9_1, ['fzb'],['l1','l3','l5'], [2,4],0)
X_test, y_test, uniques_10, day = get_feature_from_df(df_9_2, ['fzb'],['l1','l3','l5'], [2,4], 0)


X_train['y'] = y_train
X_test['y'] = y_test

# # # # X['day'] = day
X_train.to_csv('D:/BCI_Classification/20200227_level3_sameday_5freq_step250_all_fft_0.csv')
X_test.to_csv('D:/BCI_Classification/20200227_level3_sameday_5freq_step250_all_fft_2.csv')

# # # change to 0 base
# y_train = X_train['y']
# y_test = X_test['y']

# for j in X_train.columns[1:-1]:
#     fig, (ax1, ax2) = plt.subplots(1,2,figsize = (8,3))

#     for i in range(5):
#         ax1.hist(X_train.loc[y_train==i,j], label = str(i), bins = 30)
#         ax2.hist(X_test.loc[y_train==i,j], label = str(i), bins = 30)
        
#     ax1.set_title(j+'train')
#     ax2.set_title(j+'test')
    

#     ax1.set_xlim(0.2,1)  
#     ax2.set_xlim(0.2,1)  
#     plt.legend()
#     plt.show()

### data for raw data + eegnet analysis (5 class)

In [ ]:
X_list = []
y_list = []
day_list = []
for i in range(4,9):
    line = 'get_raw_data(all_fft{idx},["fzb"],["l1","l2","l3","l4","l5"], {idx})'.format(idx = i)
    try:
        [X_tmp,y_tmp,d_tmp] = eval(line)
        X_list.append(X_tmp)
        y_list.extend(y_tmp)
        day_list.extend(d_tmp)
    except (NameError, SyntaxError):
        print(line)
        
X = np.concatenate([x for x in X_list], axis = 0)
[y,target_name] = pd.factorize(y_list)

np.save('raw_data_X.npy',X)
np.save('raw_data_y.npy',y)
np.save('raw_data_target_name.npy',target_name)
np.save('raw_data_day.npy',day_list)

##### fzb, all five days, 3 levels, write to csv, fft

In [ ]:
X_train_0, y_train_0, uniques_0 = get_feature_from_df(df_0, ['fzb'],['l1','l2','l3'], 0)
X_train_1, y_train_1, uniques_1 = get_feature_from_df(df_1, ['fzb'],['l1','l2','l3'], 0)
#X_train_2, y_train_2, uniques_2 = get_feature_from_df(df_2, ['fzb'],['l1','l2','l3'], 0)
X_train_3, y_train_3, uniques_3 = get_feature_from_df(df_3, ['fzb'],['l1','l2','l4'], 0)
X_train_4, y_train_4, uniques_4 = get_feature_from_df(df_4, ['fzb'],['l1','l2','l4'], 0)
X_train_5, y_train_5, uniques_5 = get_feature_from_df(df_5, ['fzb'],['l1','l2','l4'], 0)
X_train_6, y_train_6, uniques_6 = get_feature_from_df(df_6, ['fzb'],['l1','l2','l4'], 0)
X_train_7, y_train_7, uniques_7 = get_feature_from_df(df_7, ['fzb'],['l1','l2','l4'], 0)
X_train_8, y_train_8, uniques_8 = get_feature_from_df(df_8, ['fzb'],['l1','l2','l4'], 0)

#this dataset has level 4 same as the other dataset levl 3
y_train_3[y_train_3 ==5 ] = 4
y_train_4[y_train_4 ==5 ] = 4
y_train_5[y_train_5 ==5 ] = 4
y_train_6[y_train_6 ==5 ] = 4
y_train_7[y_train_7 ==5 ] = 4
y_train_8[y_train_8 ==5 ] = 4


X = pd.concat([X_train_0, X_train_1, X_train_3, X_train_4,
               X_train_5, X_train_6, X_train_7, X_train_8], axis = 0)

y = np.concatenate([y_train_0, y_train_1, y_train_3, y_train_4, 
                    y_train_5, y_train_6, y_train_7, y_train_8], axis = 0)

day = np.concatenate([[1]* y_train_0.shape[0], [2]* y_train_1.shape[0], 
                     [3]* y_train_3.shape[0], [4]* y_train_4.shape[0], 
                     [5]* y_train_5.shape[0], [6]* y_train_6.shape[0], 
                     [7]*y_train_7.shape[0], [8]*y_train_8.shape[0]], axis = 0)



X = X[['20','21','40','41']]

X['y'] = y

X['day'] = day
X.to_csv('D:/BCI_Classification/bci_fft_3_data.csv')

# same day different time

In [ ]:
X_train, y_train, uniques_9 = get_feature_from_df(df_9_1, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_test, y_test, uniques_10 = get_feature_from_df(df_9_2, ['fzb'],['l1','l2','l3','l4','l5'], 0)



# day = np.concatenate([[1]* y_train_0.shape[0], [2]* y_train_1.shape[0], 
#                      [3]* y_train_3.shape[0], [4]* y_train_4.shape[0], 
#                      [5]* y_train_5.shape[0], [6]* y_train_6.shape[0], 
#                      [7]*y_train_7.shape[0], [8]*y_train_8.shape[0]], axis = 0)




X_train = X_train[['20','21','40','41']]
X_test = X_test[['20','21','40','41']]

X_train['y'] = y_train
X_test['y'] = y_test

# X['day'] = day
X_train.to_csv('D:/BCI_Classification/bci_fft_5_sameday_data_train.csv')
X_test.to_csv('D:/BCI_Classification/bci_fft_5_sameday_data_test.csv')


In [ ]:
X_train, y_train, uniques_9 = get_feature_from_df(df_en_9_1, ['fzb'],['l1','l2','l3','l4','l5'], 2)
X_test, y_test, uniques_10 = get_feature_from_df(df_en_9_2, ['fzb'],['l1','l2','l3','l4','l5'], 2)



# day = np.concatenate([[1]* y_train_0.shape[0], [2]* y_train_1.shape[0], 
#                      [3]* y_train_3.shape[0], [4]* y_train_4.shape[0], 
#                      [5]* y_train_5.shape[0], [6]* y_train_6.shape[0], 
#                      [7]*y_train_7.shape[0], [8]*y_train_8.shape[0]], axis = 0)




X_train = X_train[['2','4']]
X_test = X_test[['2','4']]

X_train['y'] = y_train
X_test['y'] = y_test

# X['day'] = day
X_train.to_csv('D:/BCI_Classification/bci_entropy_5_sameday_data_train.csv')
X_test.to_csv('D:/BCI_Classification/bci_entropy_5_sameday_data_test.csv')


## entropy

In [ ]:
X_train_0, y_train_0, uniques_0 = get_feature_from_df(df_en_0, ['fzb'],['l1','l2','l3'], 2)
X_train_1, y_train_1, uniques_1 = get_feature_from_df(df_en_1, ['fzb'],['l1','l2','l3'], 2)
#X_train_2, y_train_2, uniques_2 = get_feature_from_df(df_2, ['fzb'],['l1','l2','l3'], 2)
X_train_3, y_train_3, uniques_3 = get_feature_from_df(df_en_3, ['fzb'],['l1','l2','l4'], 2)
X_train_4, y_train_4, uniques_4 = get_feature_from_df(df_en_4, ['fzb'],['l1','l2','l4'], 2)
X_train_5, y_train_5, uniques_5 = get_feature_from_df(df_en_5, ['fzb'],['l1','l2','l4'], 2)
X_train_6, y_train_6, uniques_6 = get_feature_from_df(df_en_6, ['fzb'],['l1','l2','l4'], 2)
X_train_7, y_train_7, uniques_7 = get_feature_from_df(df_en_7, ['fzb'],['l1','l2','l4'], 2)
X_train_8, y_train_8, uniques_8 = get_feature_from_df(df_en_8, ['fzb'],['l1','l2','l4'], 2)

#this dataset has level 4 same as the other dataset levl 3
y_train_3[y_train_3 ==5 ] = 4
y_train_4[y_train_4 ==5 ] = 4
y_train_5[y_train_5 ==5 ] = 4
y_train_6[y_train_6 ==5 ] = 4
y_train_7[y_train_7 ==5 ] = 4
y_train_8[y_train_8 ==5 ] = 4


X = pd.concat([X_train_0, X_train_1, X_train_3, X_train_4,
               X_train_5, X_train_6, X_train_7, X_train_8], axis = 0)

y = np.concatenate([y_train_0, y_train_1, y_train_3, y_train_4, 
                    y_train_5, y_train_6, y_train_7, y_train_8], axis = 0)

day = np.concatenate([[1]* y_train_0.shape[0], [2]* y_train_1.shape[0], 
                     [3]* y_train_3.shape[0], [4]* y_train_4.shape[0], 
                     [5]* y_train_5.shape[0], [6]* y_train_6.shape[0], 
                     [7]*y_train_7.shape[0], [8]*y_train_8.shape[0]], axis = 0)

X = X[['2','4']]

X['y'] = y

X['day'] = day
X.to_csv('D:/BCI_Classification/bci_entropy_3_data.csv')

## fzb, all four days, 5 levels, write to csv, fft

In [ ]:
X_train_4, y_train_4, uniques_4 = get_feature_from_df(df_4, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_5, y_train_5, uniques_5 = get_feature_from_df(df_5, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_6, y_train_6, uniques_6 = get_feature_from_df(df_6, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_7, y_train_7, uniques_7 = get_feature_from_df(df_7, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_8, y_train_8, uniques_8 = get_feature_from_df(df_8, ['fzb'],['l1','l2','l3','l4','l5'], 0)

#this dataset has level 4 same as the other dataset levl 3


X = pd.concat([X_train_4, X_train_5, X_train_6, X_train_7, X_train_8], axis = 0)

y = np.concatenate([y_train_4, y_train_5, y_train_6, y_train_7, y_train_8], axis = 0)

day = np.concatenate([[1]* y_train_4.shape[0], [2]* y_train_5.shape[0], 
                     [3]* y_train_6.shape[0], [4]* y_train_7.shape[0], 
                      [5]*y_train_8.shape[0]],  axis = 0)

X  = X[['20','21','40','41']]

X['y'] = y
X['day'] = day
X.to_csv('D:/BCI_Classification/bci_fft_5_data.csv')

## entropy

In [ ]:
X_train_4, y_train_4, uniques_4 = get_feature_from_df(df_en_4, ['fzb'],['l1','l2','l3','l4','l5'], 2)
X_train_5, y_train_5, uniques_5 = get_feature_from_df(df_en_5, ['fzb'],['l1','l2','l3','l4','l5'], 2)
X_train_6, y_train_6, uniques_6 = get_feature_from_df(df_en_6, ['fzb'],['l1','l2','l3','l4','l5'], 2)
X_train_7, y_train_7, uniques_7 = get_feature_from_df(df_en_7, ['fzb'],['l1','l2','l3','l4','l5'], 2)
X_train_8, y_train_8, uniques_8 = get_feature_from_df(df_en_8, ['fzb'],['l1','l2','l3','l4','l5'], 2)

#this dataset has level 4 same as the other dataset levl 3


X = pd.concat([X_train_4, X_train_5, X_train_6, X_train_7, X_train_8], axis = 0)

y = np.concatenate([y_train_4, y_train_5, y_train_6, y_train_7, y_train_8], axis = 0)

day = np.concatenate([[1]* y_train_4.shape[0], [2]* y_train_5.shape[0], 
                     [3]* y_train_6.shape[0], [4]* y_train_7.shape[0], 
                      [5]*y_train_8.shape[0]],  axis = 0)

X  = X[['2','4']]

X['y'] = y
X['day'] = day
X.to_csv('D:/BCI_Classification/bci_entropy_5_data.csv')

## single day

In [ ]:
X_train_7, y_train_7, uniques_7 = get_feature_from_df(df_7, ['fzb'],['l1','l2','l3', 'l4', 'l5'], 0)
uniques_tak = uniques_7
X = X_train_7[['20','21','40','41']]
y = y_train_7


from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(cv=5, random_state=0,max_iter = 10000, multi_class = 'auto').fit(X, y)

print(clf.score(X, y))
#print('coefficient: '+str(clf.coef_))
#np.savetxt(sub+'day0_coef.csv', clf.coef_)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
X_transform = lda.fit_transform(X,y)

from matplotlib import pyplot as plt
colors = ['red', 'green', 'blue','yellow','purple']
target_names = uniques_7[np.unique(y)]

for color, i, target_name in zip(colors, np.unique(y), target_names):
    plt.scatter(X_transform[y == i, 0], X_transform[y == i, 1], color=color, alpha=.8,# lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('lda')

plt.show()

from matplotlib import pyplot as plt
colors = ['red', 'green', 'blue','yellow','purple']
target_names = uniques_7[np.unique(y)]

for color, i, target_name in zip(colors, np.unique(y), target_names):
    plt.hist(X_transform[y == i, 0], color=color, alpha=.8,# lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('LDA')

plt.show()

In [ ]:
X_train_6, y_train_6, uniques_6 = get_feature_from_df(df_7, ['fzb'],['l1','l2','l4'], 0)
uniques_tak = uniques_6
X = X_train_6[['20','21','40','41']]
y = y_train_6


from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(cv=5, random_state=0,max_iter = 10000, multi_class = 'auto').fit(X, y)

print(clf.score(X, y))
#print('coefficient: '+str(clf.coef_))
#np.savetxt(sub+'day0_coef.csv', clf.coef_)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
X_transform = lda.fit_transform(X,y)

from matplotlib import pyplot as plt
colors = ['red', 'green', 'blue','yellow']
target_names = uniques_tak[np.unique(y)]

for color, i, target_name in zip(colors, np.unique(y), target_names):
    plt.scatter(X_transform[y == i, 0], X_transform[y == i, 1], color=color, alpha=.8,# lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('lda')

plt.show()

from matplotlib import pyplot as plt
colors = ['red', 'green', 'blue','yellow','purple']
target_names = uniques_tak[np.unique(y)]

for color, i, target_name in zip(colors, np.unique(y), target_names):
    plt.hist(X_transform[y == i, 0], color=color, alpha=.8,# lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('LDA')

plt.show()

In [ ]:
X_train_7, y_train_7, uniques_7 = get_feature_from_df(df_7, ['fzb'],['l1','l2', 'l4'], 0)
uniques_tak = uniques_7
X = X_train_7[['20','21','40','41']]
y = y_train_7


from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(cv=5, random_state=0,max_iter = 10000, multi_class = 'auto').fit(X, y)

print(clf.score(X, y))
#print('coefficient: '+str(clf.coef_))
#np.savetxt(sub+'day0_coef.csv', clf.coef_)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
X_transform = lda.fit_transform(X,y)

from matplotlib import pyplot as plt
colors = ['red', 'green', 'blue','yellow','purple']
target_names = uniques_7[np.unique(y)]

for color, i, target_name in zip(colors, np.unique(y), target_names):
    plt.scatter(X_transform[y == i, 0], X_transform[y == i, 1], color=color, alpha=.8,# lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('lda')

plt.show()

from matplotlib import pyplot as plt
colors = ['red', 'green', 'blue','yellow','purple']
target_names = uniques_7[np.unique(y)]

for color, i, target_name in zip(colors, np.unique(y), target_names):
    plt.hist(X_transform[y == i, 0], color=color, alpha=.8,# lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('LDA')

plt.show()

In [ ]:
np.unique(y)

## svm, grid search

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC


X_train_0, y_train_0, uniques_0 = get_feature_from_df(df_0, ['fzb'],['l1','l2','l3'], 0)
X_train_1, y_train_1, uniques_1 = get_feature_from_df(df_1, ['fzb'],['l1','l2','l3'], 0)
#X_train_2, y_train_2, uniques_2 = get_feature_from_df(df_2, ['fzb'],['l1','l2','l3'], 0)
X_train_3, y_train_3, uniques_3 = get_feature_from_df(df_3, ['fzb'],['l1','l2','l4'], 0)
X_train_4, y_train_4, uniques_4 = get_feature_from_df(df_4, ['fzb'],['l1','l2','l4'], 0)
X_train_5, y_train_5, uniques_5 = get_feature_from_df(df_5, ['fzb'],['l1','l2','l4'], 0)
X_train_6, y_train_6, uniques_6 = get_feature_from_df(df_6, ['fzb'],['l1','l2','l4'], 0)
X_train_7, y_train_7, uniques_7 = get_feature_from_df(df_7, ['fzb'],['l1','l2','l4'], 0)

#this dataset has level 4 same as the other dataset levl 3
y_train_3[y_train_3 ==5 ] = 4
y_train_4[y_train_4 ==5 ] = 4
y_train_5[y_train_5 ==5 ] = 4
y_train_6[y_train_6 ==5 ] = 4
y_train_7[y_train_7 ==5 ] = 4

X = pd.concat([X_train_0, X_train_1, X_train_3, X_train_4, X_train_5, X_train_6, X_train_7], axis = 0)
y = np.concatenate([y_train_0, y_train_1, y_train_3,  y_train_4,  y_train_5, y_train_6, y_train_7], axis = 0)


X  = X[['20','21','40','41']]

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()
    
    
    #Classifier Confusion matrix
    from sklearn.metrics import confusion_matrix
    print('Confusion Matrix : \n' + str(confusion_matrix(y_test,y_pred)))


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import LinearSVC


X_train_0, y_train_0, uniques_0 = get_feature_from_df(df_0, ['fzb'],['l1','l2','l3'], 0)
X_train_1, y_train_1, uniques_1 = get_feature_from_df(df_1, ['fzb'],['l1','l2','l3'], 0)
#X_train_2, y_train_2, uniques_2 = get_feature_from_df(df_2, ['fzb'],['l1','l2','l3'], 0)
X_train_3, y_train_3, uniques_3 = get_feature_from_df(df_3, ['fzb'],['l1','l2','l4'], 0)
X_train_4, y_train_4, uniques_4 = get_feature_from_df(df_4, ['fzb'],['l1','l2','l4'], 0)
X_train_5, y_train_5, uniques_5 = get_feature_from_df(df_5, ['fzb'],['l1','l2','l4'], 0)
X_train_6, y_train_6, uniques_6 = get_feature_from_df(df_6, ['fzb'],['l1','l2','l4'], 0)
X_train_7, y_train_7, uniques_7 = get_feature_from_df(df_7, ['fzb'],['l1','l2','l4'], 0)

#this dataset has level 4 same as the other dataset levl 3
y_train_3[y_train_3 ==5 ] = 4
y_train_4[y_train_4 ==5 ] = 4
y_train_5[y_train_5 ==5 ] = 4
y_train_6[y_train_6 ==5 ] = 4
y_train_7[y_train_7 ==5 ] = 4

X = pd.concat([X_train_0, X_train_1, X_train_3, X_train_4, X_train_5, X_train_6, X_train_7], axis = 0)
y = np.concatenate([y_train_0, y_train_1, y_train_3,  y_train_4,  y_train_5, y_train_6, y_train_7], axis = 0)

X  = X[['20','21','40','41']]


# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

onevsrest = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train).score(X_test, y_test)
onevsone =  OneVsOneClassifier(LinearSVC(random_state=0)).fit(X_train, y_train).score(X_test, y_test)


print(onevsrest)
print(onevsone)

In [ ]:
X_train_4, y_train_4, uniques_4 = get_feature_from_df(df_4, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_5, y_train_5, uniques_5 = get_feature_from_df(df_5, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_6, y_train_6, uniques_6 = get_feature_from_df(df_6, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_7, y_train_7, uniques_7 = get_feature_from_df(df_7, ['fzb'],['l1','l2','l3','l4','l5'], 0)


#this dataset has level 4 same as the other dataset levl 3


X = pd.concat([ X_train_4, X_train_5,X_train_6,X_train_7], axis = 0)
y = np.concatenate([y_train_4,  y_train_5, y_train_6, y_train_7], axis = 0)

X  = X[['20','21','40','41']]


from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(cv=5, random_state=0,max_iter = 10000, multi_class = 'auto').fit(X, y)

print(clf.score(X, y))
#print('coefficient: '+str(clf.coef_))
#np.savetxt(sub+'day0_coef.csv', clf.coef_)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
X_transform = lda.fit_transform(X,y)

from matplotlib import pyplot as plt
colors = ['red', 'green', 'blue','yellow','purple']
target_names = uniques_5[np.unique(y)]

for color, i, target_name in zip(colors, np.unique(y), target_names):
    plt.scatter(X_transform[y == i, 0], X_transform[y == i, 1], color=color, alpha=.8,# lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('lda')

plt.show()


from matplotlib import pyplot as plt
colors = ['red', 'green', 'blue','yellow','purple']
target_names = uniques_5[np.unique(y)]

for color, i, target_name in zip(colors, np.unique(y), target_names):
    plt.hist(X_transform[y == i, 0], color=color, alpha=.8,# lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('lda')

plt.show()

In [ ]:
X_train_4, y_train_4, uniques_4 = get_feature_from_df(df_4, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_5, y_train_5, uniques_5 = get_feature_from_df(df_5, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_6, y_train_6, uniques_6 = get_feature_from_df(df_6, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_7, y_train_7, uniques_7 = get_feature_from_df(df_7, ['fzb'],['l1','l2','l3','l4','l5'], 0)


#this dataset has level 4 same as the other dataset levl 3


X = pd.concat([ X_train_4, X_train_5,X_train_6,X_train_7], axis = 0)
y = np.concatenate([y_train_4, y_train_5, y_train_6, y_train_7], axis = 0)

X  = X[['20','21','40','41']]


# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

In [ ]:
X_train_4, y_train_4, uniques_4 = get_feature_from_df(df_4, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_5, y_train_5, uniques_5 = get_feature_from_df(df_5, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_6, y_train_6, uniques_6 = get_feature_from_df(df_6, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_7, y_train_7, uniques_7 = get_feature_from_df(df_7, ['fzb'],['l1','l2','l3','l4','l5'], 0)


#this dataset has level 4 same as the other dataset levl 3

X = pd.concat([ X_train_4, X_train_5,X_train_6,X_train_7], axis = 0)
y = np.concatenate([y_train_4, y_train_5, y_train_6, y_train_7], axis = 0)

X  = X[['20','21','40','41']]

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

onevsrest = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train).score(X_test, y_test)
onevsone =  OneVsOneClassifier(LinearSVC(random_state=0)).fit(X_train, y_train).score(X_test, y_test)


print(onevsrest)
print(onevsone)

In [ ]:
#no training_5

In [ ]:
X_train_0, y_train_0, uniques_0 = get_feature_from_df(df_0, ['fzb'],['l1','l2','l3'], 0)
X_train_1, y_train_1, uniques_1 = get_feature_from_df(df_1, ['fzb'],['l1','l2','l3'], 0)
#X_train_2, y_train_2, uniques_2 = get_feature_from_df(df_2, ['fzb'],['l1','l2','l3'], 0)
X_train_3, y_train_3, uniques_3 = get_feature_from_df(df_3, ['fzb'],['l1','l2','l4'], 0)
X_train_4, y_train_4, uniques_4 = get_feature_from_df(df_4, ['fzb'],['l1','l2','l4'], 0)
#X_train_5, y_train_5, uniques_5 = get_feature_from_df(df_5, ['fzb'],['l1','l2','l4'], 0)
X_train_6, y_train_6, uniques_6 = get_feature_from_df(df_6, ['fzb'],['l1','l2','l4'], 0)
X_train_7, y_train_7, uniques_7 = get_feature_from_df(df_7, ['fzb'],['l1','l2','l4'], 0)

#this dataset has level 4 same as the other dataset levl 3
y_train_3[y_train_3 ==5 ] = 4
y_train_4[y_train_4 ==5 ] = 4
#y_train_5[y_train_5 ==5 ] = 4
y_train_6[y_train_6 ==5 ] = 4
y_train_7[y_train_7 ==5 ] = 4

X = pd.concat([X_train_0, X_train_1, X_train_3, X_train_4, X_train_6, X_train_7], axis = 0)
y = np.concatenate([y_train_0, y_train_1, y_train_3,  y_train_4, y_train_6, y_train_7], axis = 0)

X = X[['20','21','40','41']]

from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(cv=5, random_state=0,max_iter = 10000, multi_class = 'auto').fit(X, y)

print(clf.score(X, y))
#print('coefficient: '+str(clf.coef_))
#np.savetxt(sub+'day0_coef.csv', clf.coef_)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
X_transform = lda.fit_transform(X,y)

from matplotlib import pyplot as plt
colors =  ['red', 'green', 'blue','yellow','purple']
target_names = uniques_6[np.unique(y)]

for color, i, target_name in zip(colors, np.unique(y), target_names):
    plt.scatter(X_transform[y == i, 0], X_transform[y == i, 1], color=color, alpha=.8,# lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('lda')

plt.show()

from matplotlib import pyplot as plt
colors = ['red', 'green', 'blue','yellow','purple']
target_names = uniques_6[np.unique(y)]

for color, i, target_name in zip(colors, np.unique(y), target_names):
    plt.hist(X_transform[y == i, 0], color=color, alpha=.8,# lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('lda')

plt.show()


In [ ]:
X_train_4, y_train_4, uniques_4 = get_feature_from_df(df_4, ['fzb'],['l1','l2','l3','l4','l5'], 0)
#X_train_5, y_train_5, uniques_5 = get_feature_from_df(df_5, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_6, y_train_6, uniques_6 = get_feature_from_df(df_6, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_7, y_train_7, uniques_7 = get_feature_from_df(df_7, ['fzb'],['l1','l2','l3','l4','l5'], 0)


#this dataset has level 4 same as the other dataset levl 3
y_train_0[y_train_0 ==4 ] = 5
y_train_1[y_train_1 ==4 ] = 5



X = pd.concat([X_train_0, X_train_1, X_train_3, X_train_4, X_train_6,X_train_7], axis = 0)
y = np.concatenate([y_train_0, y_train_1, y_train_3,  y_train_4, y_train_6, y_train_7], axis = 0)

X  = X[['20','21','40','41']]


from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(cv=5, random_state=0,max_iter = 10000, multi_class = 'auto').fit(X, y)

print(clf.score(X, y))
#print('coefficient: '+str(clf.coef_))
#np.savetxt(sub+'day0_coef.csv', clf.coef_)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
X_transform = lda.fit_transform(X,y)

from matplotlib import pyplot as plt
colors = ['red', 'green', 'blue','yellow','purple']
target_names = uniques_6[np.unique(y)]

for color, i, target_name in zip(colors, np.unique(y), target_names):
    plt.scatter(X_transform[y == i, 0], X_transform[y == i, 1], color=color, alpha=.8,# lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('lda')

plt.show()

from matplotlib import pyplot as plt
colors = ['red', 'green', 'blue','yellow','purple']
target_names = uniques_5[np.unique(y)]

for color, i, target_name in zip(colors, np.unique(y), target_names):
    plt.hist(X_transform[y == i, 0], color=color, alpha=.8,# lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('lda')

plt.show()

In [ ]:
X_train_4, y_train_4, uniques_4 = get_feature_from_df(df_4, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_5, y_train_5, uniques_5 = get_feature_from_df(df_5, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_6, y_train_6, uniques_6 = get_feature_from_df(df_6, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_7, y_train_7, uniques_7 = get_feature_from_df(df_7, ['fzb'],['l1','l2','l3','l4','l5'], 0)


X_train = X_train_4[['20','21','40','41']]
y_train = y_train_4



X_test  =  X_train_6[['20','21','40','41']]
y_test = y_train_6



from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train,y_train)

print(lda.score(X_test, y_test))

X_train_transform = lda.transform(X_train)
X_test_transform = lda.transform(X_test)


from matplotlib import pyplot as plt
colors = ['red', 'green', 'blue','yellow','purple']
target_names = uniques_4[np.unique(y_train)]

for color, i, target_name in zip(colors, np.unique(y_train), target_names):
    plt.hist(X_train_transform[y_train == i, 0], color=color, alpha=.8,# lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('lda_train')
plt.show()




from matplotlib import pyplot as plt
colors = ['red', 'green', 'blue','yellow','purple']
target_names = uniques_4[np.unique(y)]

for color, i, target_name in zip(colors, np.unique(y_test), target_names):
    plt.hist(X_test_transform[y_test == i, 0], color=color, alpha=.8,# lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('lda_test')
plt.show()


In [ ]:
from matplotlib import pyplot as plt
colors = ['red', 'green', 'blue','yellow']
target_names = ['20','21','40','41']



In [ ]:
for color, target_name in zip(colors, target_names):
    plt.hist(X[y==2][target_name], bins = 30, label=target_name ,  alpha=.8,)

plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('level 1')
plt.xlim((0, 0.8))
plt.show()

In [ ]:
for color, target_name in zip(colors, target_names):
    plt.hist(X[y==3][target_name], bins = 30, label=target_name ,  alpha=.8,)

plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('level 2')
plt.xlim((0, 0.8))

plt.show()

In [ ]:
for color, target_name in zip(colors, target_names):
    plt.hist(X[y==4][target_name], bins = 30, label=target_name ,  alpha=.8,)

plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('level 3')
plt.xlim((0, 0.8))

plt.show()

In [ ]:
for color, target_name in zip(colors, target_names):
    plt.hist(X[y==5][target_name], bins = 30, label=target_name ,  alpha=.8,)

plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('level 4')
plt.xlim((0, 0.8))

plt.show()

In [ ]:
for color, target_name in zip(colors, target_names):
    plt.hist(X[y==6][target_name], bins = 30, label=target_name ,  alpha=.8,)

plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('level 5')
plt.xlim((0, 0.8))

plt.show()

In [ ]:
X_train_4, y_train_4, uniques_4 = get_feature_from_df(df_4, ['fzb'],['l1','l2','l3','l4','l5'], 0)
#X_train_5, y_train_5, uniques_5 = get_feature_from_df(df_5, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_6, y_train_6, uniques_6 = get_feature_from_df(df_6, ['fzb'],['l1','l2','l3','l4','l5'], 0)
X_train_7, y_train_7, uniques_7 = get_feature_from_df(df_7, ['fzb'],['l1','l2','l3','l4','l5'], 0)




X = pd.concat([X_train_4, X_train_6,X_train_7], axis = 0)
y = np.concatenate([y_train_4, y_train_6, y_train_7], axis = 0)

X = X[['20', '21', '40','41']]


def oraculo(X, y, model, params={}):
      (X_train, X_test, y_train, y_test) = train_test_split(X, y,          
                                           test_size=.2, stratify=y,        
                                           random_state= 3001)
      pipeline = Pipeline([('column', StandardScaler()),
                          ('model', model)])
      print('Estimador: ', model)
      grid = GridSearchCV(pipeline, params, 
                          scoring='neg_mean_absolute_error', 
                          n_jobs=-1, cv=3)
      grid.fit(X_train, y_train)
      pred = grid.best_estimator_.predict(X_test)
      print('Mean Absolute Error: %1.4f' %    
            (metrics.mean_absolute_error(y_test, pred)))
      print('Accuracy: %1.4f\n' % 
            (metrics.accuracy_score(y_test,   
             np.round(pred).astype(int))))
      print(metrics.classification_report(y_test,  
             np.round(pred).astype(int)))
      print('\nDone!\n\n')



models = [LinearRegression(), LogisticRegression(),
          LogisticIT(), LogisticAT()]
params = [{},{'model__max_iter': [100], 'model__C': [1.0]},   
          {'model__max_iter': [100], 'model__alpha': [1.0]},
          {'model__max_iter': [100], 'model__alpha': [1.0]}]

for m,p in zip(models, params):
         oraculo(X,
                 y, m, p)